# ข้อมูลที่เป็น category และข้อมูลที่เป็น ordinal

**จุดประสงค์การเรียนรู้**

1. ทราบว่าประเภทข้อมูลมี 3 ประเภทคือ numerical, categorical และ ordinal และทราบความแตกต่างของข้อมูลแต่ละชนิด
2. สามารถแปลงข้อมูลประเภท categorical และ ordinal ให้เป็นตัวเลขได้ โดยใช้เครื่องมือในการแปลงของ Scikit-learn ได้แก่ `OneHotEncoder`, `OrdinalEncoder` และ `ColumnTransformer`


ในบทที่ผ่านมา ข้อมูลที่เป็นตัวเลข (Numerical data) เช่นข้อมูล sepal length, sepal width, petal length, และ petal width ของดอกไอริส สามารถใช้ในการเทรนโมเดลได้สะดวก เนื่องจากตัวโมเดลก็คือสมการทางคณิตศาสตร์ จึงเป็นธรรมชาติที่จะนำไปบวกลบค่าเหล่านั้นได้

อย่างไรก็ตาม ในส่วนของข้อมูลสายพันธุ์ดอกไอริส (`Iris-setosa`, `Iris-virginica`, และ`Iris-versicolor`) จัดเป็นข้อมูลประเภท categorical (หรือเรียกอีกอย่างว่า nominal)  ซึ่งโดยทั่วไป ข้อมูลประเภท categorical นี้ เป็นข้อมูลที่**ไม่มีลำดับของค่า** มัน มักถูกเก็บค่าในรูป string อย่างเช่นกรณีของสายพันธุ์ดอกที่เป็นไปได้ 3 แบบ แต่ก็มีบ่อยครั้งที่พบเห็นเป็นตัวเลข เช่นเลขห้อง 34, 201, หรือ 555 เป็นต้น เนื่องจากค่าของเลขห้องไม่มีลำดับมากน้อย

นอกจากข้อมูลประเภท numerical และข้อมูลประเภท categorical แล้ว ยังมีข้อมูลอีกประเภทหนึ่งคือ ข้อมูลประเภท ordinal ที่มีลักษณะคล้าย ๆ กับประเภท categorical แต่ค่าของมันมีลำดับมาเกี่ยวข้อง ยกตัวอย่างเช่น ข้อมูลวุฒิการศึกษา ระดับประถม มัธยม มหาวิทยาลัย ก็อาจเรียงจากน้อยไปมากได้เป็น ประถม < มัธยม < มหาวิทยาลัย เป็นต้น






สำหรับตัวอย่างของข้อมูล ordinal เพิ่มเติม ให้พิจารณาชุดข้อมูล [Car Evaluation](https://archive.ics.uci.edu/dataset/19/car+evaluation) ซึ่งเป็นชุดข้อมูลเกี่ยวกับการประเมินค่ารถ

In [154]:
%pip install ucimlrepo

In [155]:
from ucimlrepo import fetch_ucirepo
car_evaluation = fetch_ucirepo(id=19)   # fetch dataset
X = car_evaluation.data.features.copy() # data (as pandas dataframes)
y = car_evaluation.data.targets

# บรรทัดด้านล่างนี้ แก้ค่า 5more ในคอลัมน์ doors เพียงเพื่อจะใช้เป็นตัวอย่างของ Numerical data
X.replace({'doors':'5more'}, 5, inplace=True)
X['doors'] = X['doors'].astype(int)

ชุดข้อมูล Car Evaluation มีตัวแปรดังต่อไปนี้

|คอลัมน์| ความหมาย | ค่าที่เป็นไปได้ |
|:--|:--|:-- |
|buying|	ราคาซื้อ | vhigh, high, med, low |
|maint | ค่าบำรุงรักษา		| vhigh, high, med, low |
|doors | จำนวนประตูรถ	| 2, 3, 4, 5 |
|persons | จำนวนผู้โดยสาร  | 2, 4, more |
|lug_boot	| ความจุท้ายรถ  | small, med, big |
|safety| ประมาณการความปลอดภัยของรถ  | low, med, high |
|class| ระดับการประเมินซึ่งแสดงถึง Car's Acceptability  | unacceptable, acceptable, good, very good |

จากตาราง จะเห็นว่า เป็นข้อมูลประเภท ordinal ทั้งหมดทุกคอลัมน์ ยกเว้น คอลัมน์ `doors` ที่เป็นประเภท numerical

## การแปลงข้อมูลให้เป็นตัวเลข

สามารถทำได้ 2 แบบคือ

1. แปลงเป็นตัวเลข 0, 1, 2, ...
2. แปลงด้วยตัวแปร dummy หรือเรียกว่าการแปลงแบบ one-hot



### การแปลงข้อมูลให้เป็นตัวเลข 0, 1, 2, ...

ทำโดยใช้ `OrdinalEncoder` [(คู่มือ)](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html) ซึ่งการทำงานจะเหมือนกับ `LabelEncoder` ที่เคยใช้ในคาบที่ผ่านมา เพียงแต่ `OrdinalEncoder` แปลงค่าได้ทีละหลายคอลัมน์พร้อมกัน ส่วน `LabelEncoder` แปลงได้เพียงทีละคอลัมน์

รูปแบบการใช้งาน `OrdinalEncoder` เป็นดังนี้

```python
from sklearn.preprocessing import OrdinalEncoder
ord = OrdinalEncoder()   # สร้างตัวแปลงเปล่า ๆ ขึ้นมา
ord.fit(X)               # ส่งข้อมูลให้ตัวแปลงเพื่อประเมิน
X_new = ord.transform(X) # หรือใช้ X_new = orden.fit_transform(X) จบในตัวเดียว

```

นอกจากนี้ ในการทำให้ Scikit-learn ให้ผลลัพธ์เป็น Pandas แทนที่จะเป็น Numpy ก็อาจใช้คำสั่ง
```python
ord.set_output(transform='pandas')  
```

In [192]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
ord = OrdinalEncoder()
ord.set_output(transform='pandas')  # บรรทัดนี้ เป็นการระบุให้ Scikit-learn ให้ผลลัพธ์เป็น DataFrame ของ Pandas แทนที่จะเป็น Numpy
X_new = ord.fit_transform(X)
X_new


,buying,maint,doors,persons,lug_boot,safety
0,3.0,3.0,0.0,0.0,2.0,1.0
1,3.0,3.0,0.0,0.0,2.0,2.0
2,3.0,3.0,0.0,0.0,2.0,0.0
3,3.0,3.0,0.0,0.0,1.0,1.0
4,3.0,3.0,0.0,0.0,1.0,2.0
...,...,...,...,...,...,...
1723,1.0,1.0,3.0,2.0,1.0,2.0
1724,1.0,1.0,3.0,2.0,1.0,0.0
1725,1.0,1.0,3.0,2.0,0.0,1.0
1726,1.0,1.0,3.0,2.0,0.0,2.0


จากผลลัพธ์ที่ได้ ค่าในแต่ละคอลัมน์ได้ถูกแทนค่าเป็นตัวเลข โดยเริ่มนับตามลำดับจาก 0, 1, 2, ... ตามลำดับ ตามจำนวนค่าที่เป็นไปได้ของแต่ละคอลัมน์ ซึ่งเราอาจย้อนดูว่า ตัวเลขแต่ละตัวมีความหมายว่าอย่างไร โดยดูจาก

In [157]:
ord.feature_names_in_  # ชื่อคอลัมน์ที่เป็นอินพุต

array(['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'],
      dtype=object)

In [158]:
ord.categories_ # ค่าในแต่ละคอลัมน์

[array(['high', 'low', 'med', 'vhigh'], dtype=object),
 array(['high', 'low', 'med', 'vhigh'], dtype=object),
 array([2, 3, 4, 5]),
 array(['2', '4', 'more'], dtype=object),
 array(['big', 'med', 'small'], dtype=object),
 array(['high', 'low', 'med'], dtype=object)]

จาก `ord.feature_names_in_` และ `ord.categories_` จะเห็นว่า คอลัมน์เช่น คอลัมน์ `buying` ที่มีค่า `'high'`, `'low'`, `'med'`, และ `'vhigh'` จะถูกแทนด้วย 0, 1, 2, และ 3 ตามลำดับของ index ของ array

อย่างไรก็ตาม ตัวเลขที่กำกับไม่ถูกต้องนัก เพราะ Scikit-learn ได้กำหนดหมายเลขประจำตัวให้แต่ละค่า ตามลำดับก่อนหลังที่พบในตาราง

สมมติว่า จริง ๆ แล้ว เราต้องการให้กำหนดค่า ดังนี้

```
low=0, med=1, high=2, vhigh=3
```

ก็สามารถทำได้ การแจ้งให้ `OrdinalEncoder` ได้ดังนี้

```python
ord = OrdinalEncoder(categories=ListของListที่แสดงลำดับ)
```


In [193]:
ord = OrdinalEncoder(categories=
     [['low','med','high','vhigh'],  # ลำดับของค่าในคอลัมน์ buying
      ['low','med','high','vhigh'],  # ลำดับของค่าในคอลัมน์ maint
      [2, 3, 4, 5],                  # ลำดับของค่าในคอลัมน์ doors
      ['2', '4', 'more'],            # ลำดับของค่าในคอลัมน์ persons
      ['small','med','big'],         # ลำดับของค่าในคอลัมน์ lug_boot
      ['low','med','high']]          # ลำดับของค่าในคอลัมน์ safety
)
ord.set_output(transform='pandas')
X_new = ord.fit_transform(X)
X_new

,buying,maint,doors,persons,lug_boot,safety
0,3.0,3.0,0.0,0.0,0.0,0.0
1,3.0,3.0,0.0,0.0,0.0,1.0
2,3.0,3.0,0.0,0.0,0.0,2.0
3,3.0,3.0,0.0,0.0,1.0,0.0
4,3.0,3.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...
1723,0.0,0.0,3.0,2.0,1.0,1.0
1724,0.0,0.0,3.0,2.0,1.0,2.0
1725,0.0,0.0,3.0,2.0,2.0,0.0
1726,0.0,0.0,3.0,2.0,2.0,1.0


In [194]:
ord.categories_  # ได้ว่า ค่าของแต่ละคอลัมน์ มีลำดับที่ถูกต้องตามต้องการ

[array(['low', 'med', 'high', 'vhigh'], dtype=object),
 array(['low', 'med', 'high', 'vhigh'], dtype=object),
 array([2, 3, 4, 5]),
 array(['2', '4', 'more'], dtype=object),
 array(['small', 'med', 'big'], dtype=object),
 array(['low', 'med', 'high'], dtype=object)]

### การแปลงข้อมูลให้เป็นตัวเลขโดยใช้ OneHotEncoder

รูปแบบการใช้งาน `OneHotEncoder` มีการใช้งานเหมือนกับ `OrdinalEncoder` ดังนี้

```python
from sklearn.preprocessing import OneHotEncoder
one = OneHotEncoder()   # สร้างตัวแปลงเปล่า ๆ ขึ้นมา
one.fit(X)               # ส่งข้อมูลให้ตัวแปลงเพื่อประเมิน
X_new = one.transform(X) # หรือใช้ X_new = one.fit_transform(X) จบในตัวเดียว

```

โดยสิ่งที่ `OneHotEncoder` จะทำก็คือ ทำให้คอลัมน์ใด ๆ 1 คอลัมน์ เพิ่มจำนวนเป็น K คอลัมน์ ที่เป็นคอลัมน์ที่มีแต่ค่า 0 หรือ 1 เท่านั้น โดย K = จำนวนค่าที่เป็นไปได้ทั้งหมดของคอลัมน์นั้น ยกตัวอย่างเช่น

พิจารณาคอลัมน์ `buying` ที่มีค่าที่เป็นไปได้คือ `high`, `low` , `med` และ `vhigh` (K=4) เมื่อแปลงให้แบบ one-hot ก็จะได้เป็น

| ค่า `buying` | รูป one-hot `[high, low, med, vhigh]`  |
|:--|:--|
| `high` | `[1,0,0,0]` |
| `low` | `[0,1,0,0]` |
| `med` | `[0,0,1,0]` |
| `vhigh` | `[0,0,0,1]` |

ซึ่งรูป one-hot จะมีค่าเท่ากับ 1 ได้แค่ตำแหน่งเดียวคือตำแหน่งที่ตรงกับค่าของ buying นอกจากนั้น one-hot จะเป็นศูนย์ทั้งหมด

จากรูป one-hot `[high, low, med, vhigh]` ข้างต้น ทั้ง 4 ค่าจะถูกแตกเป็น 4 คอลัมน์แยกจากกัน โดยคอลัมน์ใหม่ จะถูกตั้งชื่อคอลัมน์ในรูป `<ชื่อคอลัมน์ตั้งต้น>_<ชื่อของค่าที่one-hot=1>` ซึ่งจะได้ผลลัพธ์ที่มีคอลัมน์ชื่อ `buying_high`,	`buying_low`,	`buying_med`,	และ `buying_vhigh`

ขอให้พิจารณาโค้ดด้านล่างนี้ประกอบ

In [195]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(sparse_output=False)
onehot.set_output(transform='pandas')
X_new = onehot.fit_transform(X)
X_new

,buying_high,buying_low,buying_med,buying_vhigh,maint_high,maint_low,maint_med,maint_vhigh,doors_2,doors_3,...,doors_5,persons_2,persons_4,persons_more,lug_boot_big,lug_boot_med,lug_boot_small,safety_high,safety_low,safety_med
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1723,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1724,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1725,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
1726,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0


> **Note:** กำหนด `sparse_output=False` ให้ `OneHotEncoder` เพื่อความสะดวกในการแสดงผลเท่านั้น ในการใช้งานจริงที่ไม่ต้องการพิมพ์ตารางออกทางจอภาพ การใช้ `sparse_output=True` (ค่า default) จะช่วยประหยัดหน่วยความจำ เนื่องจากมีการเก็บข้อมูลในลักษณะที่เรียกว่า "Sparse matrix"

## การใช้ OrdinalEncoder และ OneHotEncoder

ตัวอย่างที่ผ่านมา เป็นการแสดงวิธีการใช้ `OrdinalEncoder` และ `OneHotEncoder` กับทั้งชุดข้อมูล (`X`) อย่างไรก็ตาม ในทางปฏิบัติ เรามักจะเลือกใช้การแปลงดังนี้

1. หากข้อมูลเป็นประเภท ordinal ซึ่งค่ามาก/น้อยของข้อมูลมีความหมาย  จึงเลือกใช้ `OrdinalEncoder`
2. หากข้อมูลเป็นประเภท categorical จะเลือกใช้ `OrdinalEncoder` หรือ `OneHotEncoder` ขึ้นกับว่าโมเดลที่จะฝึกสอน เป็นโมเดลอะไร
 - Linear models (`LinearRegression/LogisticRegression`) มักเลือกใช้ `OneHotEncoder` จะดีที่สุด เพราะโมเดลจะเห็นค่าชัดเจนในทุกมิติของข้อมูลประเภท categorical นั้น (มี weight ของโมเดลที่เพิ่มขึ้นตามจำนวนคอลัมน์ที่เพิ่มขึ้น)
 - โมเดลจำพวกต้นไม้ (Tree-based models) มักเลือกใช้ `OrdinalEncoder` จะดีที่สุด

สำหรับในหัวข้อนี้ จะยกตัวอย่างการแปลงชนิดข้อมูลที่สามารถระบุได้ว่า จะเลือกใช้  `OrdinalEncoder` หรือ `OneHotEncoder` กับคอลัมน์ใด โดยใช้ `ColumnTransformer` มาครอบอีกที ซึ่งมีรูปแบบการใช้ ดังนี้

<figure>
<img src="https://raw.githubusercontent.com/INRIA/scikit-learn-mooc/main/figures/api_diagram-columntransformer.svg" width=80%>
<figcaption>ที่มารูป scikit-learn MOOC by scikit-learn developers (https://inria.github.io/scikit-learn-mooc/)</figcaption>
</figure>

```python
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([
       (ชื่อเล่น, OneHotEncoder, รายชื่อคอลัมน์ที่ทำ OneHotEncoder),    # คือ Transformer A ในรูปข้างบน
       (ชื่อเล่น, OrdinalEncoder, รายชื่อคอลัมน์ที่ทำ OrdinalEncoder),  # คือ Transformer B ในรูปข้างบน                          
    ],                        
    remainder='passthrough', # คอลัมน์ที่ไม่มีในรายชื่อที่ให้แปลง ก็ไม่ต้องทำอะไร
    verbose_feature_names_out=False # ไม่ต้องเอาชื่อเล่นที่กำหนด ไปเป็น prefix ชื่อคอลัมน์ของผลลัพธ์                      
)
ct.fit(X)               # ส่งข้อมูลให้ตัวแปลงเพื่อประเมิน
X_new = ct.transform(X) # หรือใช้ X_new = ct.fit_transform(X) จบในตัวเดียว
```

โดยในตัวอย่างด้านล่างนี้ เราต้องการแปลง ดังนี้

1. คอลัมน์ `lug_boot` แปลงด้วย `OneHotEncoder` (หมายเหตุ: จริง ๆ แล้ว `lug_boot` ควรแปลงด้วย `OrdinalEncoder` เพราะเป็นค่ามีลำดับค่าด้วย (`small`,`med`,`big`) แต่ในที่นี้ จะแสร้งแปลงเป็นแบบ one-hot เพื่อเป็นตัวอย่างเท่านั้น)
2. คอลัมน์ `buying`,`maint`,`persons`,`safety` แปลงด้วย `OrdinalEncoder` โดยมีการกำหนดลำดับค่าที่ถูกต้องให้ด้วย
3. คอลัมน์ `doors` ไม่ต้องทำอะไร เพราะเป็นข้อมูลประเภท numerical อยู่แล้ว

ซึ่งสามารถเขียนเป็นโค้ดได้ ดังนี้

In [196]:
from sklearn.compose import ColumnTransformer

# สร้างตัวแปลง OneHotEncoder เปล่า ๆ ขึ้นมาเช่นเคย
one = OneHotEncoder(sparse_output=False)

# สร้างตัวแปลง OrdinalEncoder เปล่า ๆ ขึ้นมาเช่นเคย พร้อมระบุลำดับค่าของคอัมน์ที่จะแปลงใหู้กต้อง
ord = OrdinalEncoder(
     categories=
      [['low','med','high','vhigh'],  # ลำดับของค่าในคอลัมน์ buying
       ['low','med','high','vhigh'],  # ลำดับของค่าในคอลัมน์ maint
       ['2', '4', 'more'],            # ลำดับของค่าในคอลัมน์ persons
       ['low','med','high']]          # ลำดับของค่าในคอลัมน์ safety
)

# สร้าง ColumnTransformer มาครอบ OneHotEncoder และ OrdinalEncoder ที่สร้างไว้แล้ว
ct1 = ColumnTransformer([  # 'my_hot' และ 'my_ord' เป็นเพียง prefix ที่ตั้งเอง
       ('my_hot', one, ['lug_boot']),
       ('my_ord', ord, ['buying','maint','persons','safety']),
    ],
    remainder='passthrough', # ให้ปล่อยผ่านคอลัมน์ที่เหลือ (คือ doors) มาโดยไม่ต้องทำอะไร
    verbose_feature_names_out=False  # ไม่ต้องแก้ไขชื่อคอลัมน์ของผลลัพธ์ที่ได้
)
ct1.set_output(transform='pandas')
X_new = ct1.fit_transform(X)
X_new

,lug_boot_big,lug_boot_med,lug_boot_small,buying,maint,persons,safety,doors
0,0.0,0.0,1.0,3.0,3.0,0.0,0.0,2
1,0.0,0.0,1.0,3.0,3.0,0.0,1.0,2
2,0.0,0.0,1.0,3.0,3.0,0.0,2.0,2
3,0.0,1.0,0.0,3.0,3.0,0.0,0.0,2
4,0.0,1.0,0.0,3.0,3.0,0.0,1.0,2
...,...,...,...,...,...,...,...,...
1723,0.0,1.0,0.0,0.0,0.0,2.0,1.0,5
1724,0.0,1.0,0.0,0.0,0.0,2.0,2.0,5
1725,1.0,0.0,0.0,0.0,0.0,2.0,0.0,5
1726,1.0,0.0,0.0,0.0,0.0,2.0,1.0,5


จากผลลัพธ์ของ `ColumnTransformer` จะเห็นว่า
- คอลัมน์ `lug_boot` แตกเป็น 3 คอลัมน์ คือ `lug_boot_big`,	`lug_boot_med`,	และ `lug_boot_small`
- คอลัมน์ `buying`,	`maint`,	`persons`,	`safety` ถูกแปลงเป็นประเภท ordinal
- คอลัมน์ `doors` ไม่ถูกแตะเลย

จบ.

In [190]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()

# ct2 = ColumnTransformer([
#        ('scaler', scaler, ['doors','buying','maint','persons','safety']),
#     ],
#     remainder='passthrough',
#     verbose_feature_names_out=False
# )
# ct2.set_output(transform='pandas')
# X_new2 = ct2.fit_transform(X_new)
# X_new2

,doors,buying,maint,persons,safety,lug_boot_big,lug_boot_med,lug_boot_small
0,-1.341641,1.341641,1.341641,-1.224745,-1.224745,0.0,0.0,1.0
1,-1.341641,1.341641,1.341641,-1.224745,0.000000,0.0,0.0,1.0
2,-1.341641,1.341641,1.341641,-1.224745,1.224745,0.0,0.0,1.0
3,-1.341641,1.341641,1.341641,-1.224745,-1.224745,0.0,1.0,0.0
4,-1.341641,1.341641,1.341641,-1.224745,0.000000,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...
1723,1.341641,-1.341641,-1.341641,1.224745,0.000000,0.0,1.0,0.0
1724,1.341641,-1.341641,-1.341641,1.224745,1.224745,0.0,1.0,0.0
1725,1.341641,-1.341641,-1.341641,1.224745,-1.224745,1.0,0.0,0.0
1726,1.341641,-1.341641,-1.341641,1.224745,0.000000,1.0,0.0,0.0
